# MNIST Exercise

#### 1. The MNIST
MNIST is a simple computer vision dataset. It consists of images of handwritten digits and labels for each image. You may meet warning after running the first line code. It doesn't impact the result of exercise.

#### 1.1 MNIST Data load

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#### 1.2 Shape of dataset

In [ ]:
import numpy as np
print("Training Data X Shape is", mnist.train.images.shape)
print("Training Data Y Shape is", mnist.train.labels.shape)
print("Test Data X Shape is", mnist.test.images.shape)
print("Test Data Y Shape is", mnist.test.labels.shape)
## print(mnist.train.__class__)
## print(mnist.test.__class__)

#### 1.3 Data Visualization

If image doesn't come out, please run below code again.

In [ ]:
import matplotlib.pyplot as plt
for i in range(1,9):
    plt.subplot(340+i)
    plt.imshow(mnist.train.images[30+i*5000].reshape(28,28),cmap='gray_r')
    plt.tight_layout()

#### 2. Import TensorFlow

In [ ]:
import tensorflow as tf
## tf.__version__ 

## MNIST with Softmax Regression
The first half of exercise refers to TensorFlow tutorial. https://www.tensorflow.org/versions/r1.4/get_started/mnist/beginners

#### 3. Placeholders
We'll input when we ask TensorFlow to run a computation. We want to be able to input any number of MNIST images, each flattened into a 784-dimensional vector. We represent this as a 2-D tensor of floating-point numbers, with a shape [None, 784]. (Here None means that a dimension can be of any length.)

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])

#### 4. Variables: Weights and biases
A Variable is a modifiable tensor that lives in TensorFlow's graph of interacting operations.

In [ ]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

#### 5. Model Implementation
1. Multiply x by W
2. Add b
3. Apply tf.nn.softmax

In [ ]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

## Training

#### 6-1. Cross-entropy placeholder 
new placeholder to input the correct answers

In [ ]:
y_ = tf.placeholder(tf.float32, [None, 10])

#### 6-2. Cross-entropy function

API reference
- tf.reduce_mean https://www.tensorflow.org/api_docs/python/tf/reduce_mean
- tf.nn.softmax_cross_entropy_with_logits_v2 https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits_v2

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), axis=[1]))

## cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y)) 

#### 6-3. Optimization algorithm

- GradientDescentOptimizer https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer
- Default learning rate = 0.5

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#### 7-1. Launch model
in an InteractiveSession

In [ ]:
sess = tf.InteractiveSession()

#### 7-2. Initialize the variables

In [ ]:
tf.global_variables_initializer().run()

#### 7-3. Train

Load 100 training examples in each 1000 training iteration

```def next_batch(self, batch_size, fake_data=False, shuffle=True):```
Return the next `batch_size` examples from this data set.

In [ ]:
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

## Evaluate the Model

#### 8. Comparison prediction and the truth

- tf.argmax is an extremely useful function which gives you the index of the highest entry in a tensor along some axis
- tf.equal to check if our prediction matches the truth

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

That gives us a list of booleans. To determine what fraction are correct, we cast to floating point numbers and then take the mean. For example, [True, False, True, True] would become [1,0,1,1] which would become 0.75

- tf.cast https://www.tensorflow.org/api_docs/python/tf/cast

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

## MNIST with CNN

The second half of exercise refers to TensorFlow tutorial with minor modification. https://www.tensorflow.org/versions/r1.4/get_started/mnist/pros

![CNN](https://github.com/baemins/SNU_CS_M2177.004300/tree/master/images/cnnmnist.png)

Structure image from http://www.cnblogs.com/BigBallon/p/6701846.html



#### 9-1. Weight Initialization
initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients; use tf.truncated_normal

In [ ]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

#### 9-2. Convolution and Pooling

In [ ]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#### 9-3. 1st Convolutional Layer

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [ ]:
x_image = tf.reshape(x, [-1,28,28,1])

In [ ]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#### 9-4. 2nd Convolutional Layer

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#### 9-5. Densely Connected Layer

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#### 9-6. Dropout

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#### 9-7. Readout Layer

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#### 10. Train and Test the Model

In [ ]:
### Do not replace tf.nn.softmax_cross_entropy_with_logits API to v2, causing crash
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

### Original code;
### print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

### Replaced the original Tensorflow code to prevent GPU memory error
### https://stackoverflow.com/questions/39076388/tensorflow-deep-mnist-resource-exhausted-oom-when-allocating-tensor-with-shape

for i in range(10):
    testSet = mnist.test.next_batch(50)
    print("test accuracy %g"%accuracy.eval(feed_dict={ x: testSet[0], y_: testSet[1], keep_prob: 1.0}))